In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import pymc as pm
import arviz as az

Load and preprocess the penguins data set. Download [on Moodle](https://moodle.tu-darmstadt.de/pluginfile.php/1868005/mod_folder/content/0/penguins.csv?forcedownload=1).

In [4]:
penguins = pd.read_csv("penguins.csv")
# Subset to the columns needed
missing_data = penguins.isnull()[
    ["bill_length_mm", "flipper_length_mm", "sex", "body_mass_g"]
].any(axis=1)
# Drop rows with any missing data
penguins = penguins.loc[~missing_data]

## Basic model
We first fit a model with one mean and standard deviation per species.

In [5]:
# pd.categorical makes it easy to index species below
all_species = pd.Categorical(penguins["species"])

with pm.Model() as model_penguin_mass_all_species:
    # Hyperprior for μ
    μ_0 = pm.Normal("μ_0", 4000, 3000)
    σ_0 = pm.HalfStudentT("σ_0", 100, 2000)
    
    # Note the addition of the shape parameter
    σ = pm.HalfStudentT("σ", 100, 2000, shape=3)
    μ = pm.Normal("μ", 4000, 3000, shape=3)
    mass = pm.Normal("mass",
                     mu=μ[all_species.codes],
                     sigma=σ[all_species.codes],
                     observed=penguins["body_mass_g"])

    inf_data_model_penguin_mass_all_species = pm.sample()
    inf_data_model_penguin_mass_all_species = inf_data_model_penguin_mass_all_species.assign_coords({"μ_dim_0": all_species.categories,
                "σ_dim_0": all_species.categories})

# Extract posterior means
posterior_means = az.summary(inf_data_model_penguin_mass_all_species, var_names=["μ_0", "μ"])
mean_hyperprior = posterior_means.loc["μ_0", "mean"]
species_means = posterior_means.loc[["μ[0]", "μ[1]", "μ[2]"], "mean"]

# Calculate the empirical mean
empirical_mean = penguins["body_mass_g"].mean()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [μ_0, σ_0, σ, μ]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 442 seconds.


KeyError: "None of [Index(['μ[0]', 'μ[1]', 'μ[2]'], dtype='object')] are in the [index]"

In [ ]:
axes = az.plot_forest(inf_data_model_penguin_mass_all_species, var_names=["μ"], figsize=(8, 2.5))
axes[0].set_title("μ Mass Estimate: 94.0% HDI")

print(f"Estimated mean of the hyperprior (μ_0): {mean_hyperprior}")
print(f"Empirical mean of penguin body masses: {empirical_mean}")

In [ ]:
axes = az.plot_forest(inf_data_model_penguin_mass_all_species, var_names=["σ"], figsize=(8, 3))
axes[0].set_title("σ Mass Estimate: 94.0% HDI")

## Hierarchical extension